In [1]:
import pandas as pd
import requests
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval    
import scipy
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

In [2]:
df = pd.read_csv('C:/Sahith/projects/Books/GoodReads_100k_books.csv')
df.head()

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [3]:
df.isnull().sum()

author              0
bookformat       3228
desc             6772
genre           10467
img              3045
isbn            14482
isbn13          11435
link                0
pages               0
rating              0
reviews             0
title               1
totalratings        0
dtype: int64

In [4]:
df.fillna(value="",inplace=True)

In [5]:
a=[]
def collect(str):
    a.append(str.split(','))
genre_data = df['genre'].apply(collect)
a

[['History',
  'Military History',
  'Civil War',
  'American History',
  'American Civil War',
  'Nonfiction',
  'North American Hi...',
  'American History',
  'Native Americans'],
 ['Couture', 'Fashion', 'Historical', 'Art', 'Nonfiction'],
 ['Politics', 'History'],
 ['Labor', 'History'],
 [''],
 ['Business',
  'Leadership',
  'Romance',
  'Historical Romance',
  'Nonfiction',
  'Business',
  'Management',
  'Management',
  'Human Resources'],
 ['Business',
  'Leadership',
  'Business',
  'Management',
  'Romance',
  'Historical Romance'],
 ['History', 'Nonfiction'],
 [''],
 ['Religion',
  'Buddhism',
  'Philosophy',
  'Spirituality',
  'Psychology',
  'Nonfiction',
  'Religion'],
 [''],
 ['Anthropology',
  'Religion',
  'Nonfiction',
  'Academic',
  'Social Science',
  'Sociology'],
 ['Biology', 'Ecology', 'Nonfiction', 'Politics'],
 ['Psychology',
  'Nonfiction',
  'Economics',
  'Science',
  'Philosophy',
  'Self Help',
  'Self Help',
  'Personal Development',
  'Sociology',
  'Wr

In [6]:
import itertools

b = list(itertools.chain.from_iterable(a))
b = [genre for genre in b if genre]# Removing empty strings



In [7]:

empty = set()  # Initialize an empty set

for item in b:  # Iterate over the elements, not indices
    empty.add(item)  # Add each genre to the set
new_list = list(empty)
print(len(new_list))


1182


In [8]:

df['desc'] = df['desc'].fillna('')
def clean_text(text):
    text = str(text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip().lower()

df['desc'] = df['desc'].apply(clean_text)

In [9]:
df["genre"] = df["genre"].apply(lambda x: literal_eval(x) if isinstance(x, str) and x.startswith("[") else x.split(','))


In [10]:
def genre_to_string(genres):
    return " ".join(genres) if isinstance(genres, list) else genres

df["genre"] = df["genre"].apply(genre_to_string)

In [11]:
mlb=MultiLabelBinarizer()
genre_features=mlb.fit_transform(df['genre'])

In [12]:
tfidf = TfidfVectorizer(stop_words='english',max_features=5000)
desc_features=tfidf.fit_transform(df['desc'])

In [18]:
import scipy.sparse
combined_features=scipy.sparse.hstack([genre_features,desc_features]).tocsr()
combined_features

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5853297 stored elements and shape (100000, 5072)>

In [14]:
knn = NearestNeighbors(metric='cosine',algorithm='brute')
knn.fit(combined_features)


NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
def recommend_content_based(title, top_n=20):
    idx = df[df['title'] == title].index[0]
    query = combined_features[idx].reshape(1,-1)
    distances, indices = knn.kneighbors(query, n_neighbors=top_n+1)
    return df[['title', 'genre', 'desc', 'rating', 'totalratings','img','link','author']].iloc[indices[0][1:]]

In [16]:
user_preferences = {
    'user1': ['Classic', 'Sci-Fi', 'Thriller'],
    'user2': ['Mystery', 'Thriller'],
    
}


user_item_matrix = pd.DataFrame(0, index=user_preferences.keys(), columns=new_list)
for user, preferences in user_preferences.items():
    for genre in preferences:
        if genre in user_item_matrix.columns:  # Check if the genre exists in the matrix
            user_item_matrix.loc[user, genre] = 1


# Display the updated matrix
user_item_matrix

,Science Fiction,Nigeria,Harlequin Heartwarming,Fantasy Of Manners,Indian Literature,Books About Books,Alcohol,Writing,Mannerpunk,Indigenous History,...,Biology,Crime,Victorian,Bulgaria,Halloween,Spirituality,Leningrad,Love Inspired Historical,The Americas,Southern
user1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
user2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
item_similarity = cosine_similarity(user_item_matrix.T)  # Transpose to get item-item similarity
item_similarity_df = pd.DataFrame(item_similarity, index=new_list, columns=new_list)

# Function to recommend books based on user preferences
def recommend_collaborative(user, top_n=5):
    user_pref = user_preferences[user]
    similar_genres = item_similarity_df[user_pref].mean(axis=1).sort_values(ascending=False)
    recommended_books = df[df['genre'].isin(similar_genres.index)].sort_values(by='rating', ascending=False)
    return recommended_books[['title', 'genre', 'rating', 'totalratings', 'img', 'link', 'author']].head(top_n)

# Example usage
recommend_collaborative('user1')

KeyError: "['Classic', 'Sci-Fi'] not in index"

In [ ]:
def hybrid_recommend_with_collab(title, user, top_n=5):
    content_based_recs = recommend_content_based(title, 20)
    C = content_based_recs['rating'].mean()
    m = content_based_recs['totalratings'].quantile(0.9)
    
    def bayesian_rating(row, C, m):
        v = row['totalratings']
        R = row['rating']
        return (v / (v + m) * R) + (m / (v + m) * C)
    
    content_based_recs["bayesian_score"] = content_based_recs.apply(lambda row: bayesian_rating(row, C, m), axis=1)
    
    collab_recs = recommend_collaborative(user, top_n)
    
    combined_recs = pd.concat([content_based_recs, collab_recs]).drop_duplicates(subset=['title'])
    
    combined_recs = combined_recs.sort_values(by="bayesian_score", ascending=False)
    #print(combined_recs)
    img = []
    for _, row in combined_recs.head(top_n).iterrows():
        img.append({
            'title': str(row['title']),
            'image': str(row['img'])  
        })
    
    return combined_recs[['title', 'genre', 'desc', 'rating', 'totalratings', 'img', 'link', 'author']].head(top_n)

# Example usage
hybrid_recommend_with_collab('Tyneham', 'user1')

,title,genre,desc,rating,totalratings,img,link,author
34225,"Automotive Technology: Principles, Diagnosis, ...",Transport Cars,automotive technology principles diagnosis and...,4.40,47,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/12384812-autom...,James D. Halderman
67262,Competition Engine Building: Advanced Engine D...,Transport Cars,the needs of a true competition engine are qui...,4.83,6,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/14641567-compe...,John Baechtel
89403,America's Wildest Show Rods of the 1960s and 1...,Transport Cars,the 1960s and 1970s were a time of artistic ex...,4.60,5,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/16231258-ameri...,Scotty Gosson
69835,Triumph TR6,Transport Cars,,4.67,3,,https://goodreads.com/book/show/14941865-trium...,William Kimberley
68401,Cleaning Painted Surfaces: Aqueous Methods,Conservation,this book contributes significantly to the sel...,4.50,4,https://i.gr-assets.com/images/S/compressed.ph...,https://goodreads.com/book/show/1478057.Cleani...,Richard Wolbers
